# Training Tutorial

This notebook goes through parts 1-3 in the preprocessing, training, and evaluation of the images. For each part, there is a detailed notebook in their respective directories. 

In [ ]:
import sys
sys.path.insert(0,'1_preprocessing')
sys.path.insert(0,'2_trainer')
sys.path.insert(0,'3_evaluate')
from make_data import *
from input_data import *
from models import *
from training_utilities import *
from evaluate import *
from os import makedirs

## Part 1: Preprocessing

In [ ]:
input_dir = "../data/WSeTe/simulated/"
data_dirs = [str(i) for i in range(2)]
label_list = ["Se"]
parsed_dir_name='parsed_label_Se'
ftype = '.tiff'

In [ ]:
l_shape = (256,256)
stride = (128,128)
one_pickle=False
tr_bs = 200
ts_bs = 20
ones_percent = .00
tol = 0.05
show_plots=False

In [ ]:
create_augments(input_dir, data_dirs, ftype)

In [ ]:
make_data(input_dir, label_list, data_dirs, l_shape, stride, ftype,\
        parsed_dir_name=parsed_dir_name, tr_bs=tr_bs, ts_bs=ts_bs, ones_percent=ones_percent, \
        tol=tol, show_plots=show_plots, one_save=one_pickle)

In [ ]:
parsed_fn = input_dir + parsed_dir_name + "/train/train_00000.p"
check_data(parsed_fn, l_shape=l_shape) 

## Part 2: Training

In [ ]:
parent_dir = '../'
data_dir   = parent_dir + 'data/WSeTe/simulated/parsed_label_Se/'
sess_name  = 'Se'
N          = 256
k_fac      = 16
nb_classes = 2

In [ ]:
sess_dir = parent_dir + "results/" + sess_name + "/"
makedirs(sess_dir, exist_ok=True)

model_weights_fn = sess_dir + "weights.h5"
model_fn         = sess_dir + "model.json"
diagnostics_fn   = sess_dir + "diagnostics.dat"

In [ ]:
model = construct_model(N, k_fac, nb_classes, sess_dir, model_fn, model_weights_fn)
step = setup_diagnostics(diagnostics_fn)

In [ ]:
train(step, data_dir, N, nb_classes, model, diagnostics_fn, model_weights_fn)

## Part 3: Evaluation 

In [ ]:
results_dir = sess_dir
data_dir = parent_dir + "data/WSeTe/simulated/3/"


model_fn = results_dir + "model.json"                                              
model_weights_fn = results_dir + "weights.h5"
input_file =  data_dir + "input.tiff"
l_shape = (256, 256)
stride  = (64, 64)
avg = 0
plot=True
save_data=False
save_dir = "./"                                                         
prefix = "Se"

In [ ]:
%matplotlib notebook
prediction = evaluate(model_fn, model_weights_fn, input_file, l_shape, stride,
                      avg=avg, plot=plot, save_data=save_data, save_dir=save_dir, prefix=prefix)

In [ ]:
label_file_list = [data_dir + "label_{}.tiff".format("Se")]
tol=.5
nconvs = 0
r = 3
TN = 0
plot=True
save_data=False
save_dir = "./"                                                         
prefix = "Se"
verbose = True

In [ ]:
%matplotlib notebook
TP, FP, FN, TN, recall, precision, F1, bal_acc = calc_accuracy(prediction, label_file_list, tol=tol, 
                                                               nconvs=nconvs, r=r, TN=TN, plot=plot, 
                                                               save_data=save_data, save_dir=save_dir, 
                                                               prefix=prefix, verbose=verbose)